This tutorial describes how to evaluate rules that are applicable to two consecutive periods (year and quarter).

In [ ]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator 

In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile
import pickle
import re
from src import Evaluator
import logging
import data_patterns
import datetime

In [ ]:
DECIMALS = 0
RULES_PATH = join('..', 'ftk-rules')
INSTANCES_DATA_PATH = join('..','data','instances')
DATAPOINTS_PATH = join('..', 'data', 'datapoints')

### Import rules

We start with importing the (t-1)-t rules that are applicable to two consecutive periods. We import a set of rules used to evaluate year data and a set of rules for quarter data.

#### FTK_betweenperiods_JS

In [ ]:
dfr_JS = pd.read_excel(join(RULES_PATH,'FTK_betweenperiods_JS.xlsx'), engine='openpyxl')

#### FTK_betweenperiods_BEL

In [ ]:
dfr_BEL = pd.read_excel(join(RULES_PATH,'FTK_betweenperiods_BEL.xlsx'), engine='openpyxl')
dfr_BEL

### Import templates

Next we import the reporting data. We import the data of two consecutive periods. In the tutorial 'Convert XBRL-instances to CSV, HTML and pickles' the XBRL-instances are converted to pickle files per template. The pickle files are written to the data/instances folder. The rules are applicable to all tables with closed axis. We import these pickle files. When comparing two periods it can be the case that two different taxonomies are applicable. The right taxonomy has to be selected in the tutorial 'Convert XBRL-instances to CSV, HTML and pickles' to convert the XBRL-instance properly. 

The list _instances_JS_ contains the names of the folders with the converted XBRL-instance for yearly data. The list _instances_BEL_ contains the names of the folders with the converted XBRL-instance for two consecutive quarters. Finally, we also have to define the category of the insurer. The rules are set-up for each type of insurer separately.

In [ ]:
instances_JS = []
instances_BEL = []

#### FTK_betweenperiods_JS

In [ ]:
# with open(join(DATAPOINTS_PATH, 'JS.pkl'), 'rb') as handle:
#     JS = pickle.load(handle)
# dft = pd.DataFrame()
# for instance in instances_JS:
#     df_closed_axis = pd.DataFrame()
#     tables_closed_axis = []  # for listing all input tables with closed axis
#     tables = [table for table in JS 
#         if isfile(join(INSTANCES_DATA_PATH,instance,table + '.pickle'))]  # JS tables found in the specified instance path
#     for table in [table for table in tables]:  #tables:
#         if isfile(join(INSTANCES_DATA_PATH,instance, table + '.pickle')):
#             df = pd.read_pickle(join(INSTANCES_DATA_PATH,instance, table + '.pickle'))  # read dataframe
#         else:
#             continue   
#         if df.index.nlevels > 2:  # if more than 2 indexes (entity, period), then the table has an open axis
#             continue
#         else:  # closed axis
#             tables_closed_axis.append(table)  # add to relevant list
#             # Add table to dataframe with all data from closed axis tables
#             if len(df_closed_axis) == 0:  # no data yet --> copy dataframe
#                 df_closed_axis = df.copy()
#             else:  # join to existing dataframe
#                 df_closed_axis = df_closed_axis.join(df)
#     if len(dft) == 0:  # no data yet 
#         dft = df_closed_axis
#     else:  # join to existing dataframe
#         dft=dft.append(df_closed_axis)
# dft=dft.reset_index()
# numerical_columns = ['entity','period'] + [dft.columns[c] for c in range(len(dft.columns))
#                          if ((dft.dtypes[c] == 'float64') or (dft.dtypes[c] == 'int64'))]
# df_JS = dft[numerical_columns]
# df_JS['period']=df_JS['period'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d')) #convert to datetime
# df_JS.fillna(0,inplace=True)

In this tutorial we work with dummy data in order to show results

In [ ]:
with open(join('..','tests','data','demo','FTK_JS.pkl'), 'rb') as handle:
    df_JS = pickle.load(handle)
df_JS

#### S2_betweenperiods_QRS

In [ ]:
# with open(join(DATAPOINTS_PATH, 'BEL.pkl'), 'rb') as handle:
#     BEL = pickle.load(handle)
# dft = pd.DataFrame()
# for instance in instances_BEL:
#     df_closed_axis = pd.DataFrame()
#     tables_closed_axis = []  # for listing all input tables with closed axis
#     tables = [table for table in BEL 
#         if isfile(join(INSTANCES_DATA_PATH,instance,table + '.pickle'))]  # BEL tables found in the specified instance path
#     for table in [table for table in tables]:  #tables:
#         if isfile(join(INSTANCES_DATA_PATH,instance, table + '.pickle')):
#             df = pd.read_pickle(join(INSTANCES_DATA_PATH,instance, table + '.pickle'))  # read dataframe
#         else:
#             continue   
#         if df.index.nlevels > 2:  # if more than 2 indexes (entity, period), then the table has an open axis
#             continue
#         else:  # closed axis
#             tables_closed_axis.append(table)  # add to relevant list
#             # Add table to dataframe with all data from closed axis tables
#             if len(df_closed_axis) == 0:  # no data yet --> copy dataframe
#                 df_closed_axis = df.copy()
#             else:  # join to existing dataframe
#                 df_closed_axis = df_closed_axis.join(df)
#     if len(dft) == 0:  # no data yet 
#         dft = df_closed_axis
#     else:  # join to existing dataframe
#         dft=dft.append(df_closed_axis)
# dft=dft.reset_index()
# numerical_columns = ['entity','period'] + [dft.columns[c] for c in range(len(dft.columns))
#                          if ((dft.dtypes[c] == 'float64') or (dft.dtypes[c] == 'int64'))]
# df_BEL = dft[numerical_columns]
# df_BEL['period']=df_BEL['period'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d')) #convert to datetime
# df_BEL.fillna(0,inplace=True)

In this tutorial we work with dummy data in order to show results

In [ ]:
with open(join('..','tests','data','demo','FTK_BEL.pkl'), 'rb') as handle:
    df_BEL = pickle.load(handle)
df_BEL

### Evaluate rules

#### Evaluate FTK_betweenperiods_JS

In [ ]:
# Get analyze
miner = data_patterns.PatternMiner(df_patterns=dfr_JS)
miner.df_data = df_JS
miner.convert_to_time(['entity'], 'period')
miner.df_data = miner.df_data.reset_index()

results = miner.analyze()
results

#### FTK_betweenperiods_BEL

In [ ]:
# Get analyze
miner = data_patterns.PatternMiner(df_patterns=dfr_BEL)
miner.df_data = df_BEL
miner.convert_to_time(['entity'], 'period', set_year=False)
miner.df_data = miner.df_data.reset_index()

results = miner.analyze()
results